In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import plotly.express as px

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-08-10  179.479996  180.750000  177.600006  177.970001  177.729996   
       2023-08-11  177.320007  178.619995  176.550003  177.789993  177.789993   
       2023-08-14  177.970001  179.690002  177.309998  179.460007  179.460007   
       2023-08-15  178.880005  179.479996  177.050003  177.449997  177.449997   
       2023-08-16  177.130005  178.539993  176.500000  176.570007  176.570007   

                     Volume  
Ticker Date                  
AAPL   2023-08-10  54686900  
       2023-08-11  51988100  
       2023-08-14  43675600  
      

In [2]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-08-10  179.479996  180.750000  177.600006  177.970001   
1   AAPL 2023-08-11  177.320007  178.619995  176.550003  177.789993   
2   AAPL 2023-08-14  177.970001  179.690002  177.309998  179.460007   
3   AAPL 2023-08-15  178.880005  179.479996  177.050003  177.449997   
4   AAPL 2023-08-16  177.130005  178.539993  176.500000  176.570007   

    Adj Close    Volume  
0  177.729996  54686900  
1  177.789993  51988100  
2  179.460007  43675600  
3  177.449997  43622600  
4  176.570007  46964900  


In [3]:
import plotly.express as px
fig = px.line(df, x='Date', 
              y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [4]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [5]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
60  172.190001  174.934502
61  172.813000  174.946501
62  173.651001  175.118002
63  174.830000  175.272501
64  176.382001  175.357501

[65 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
195         NaN         NaN
196         NaN         NaN
197         NaN         NaN
198         NaN         NaN
199         NaN         NaN
..          ...         ...
255  128.909999  134.242500
256  128.264999  133.839999
257  127.492999  133.499999
258  128.151999  133.077999
259  128.976999  132.648000

[65 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
69          NaN         NaN
..          ...         ...
125  338.083002  334.15

In [6]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [7]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

In [8]:
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')


# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 title='Correlation between Apple and Microsoft')



fig.show()
